In [1]:
import pandas as pd

In [2]:
allowed_interactions_source = pd.read_excel("eval/41467_2021_23666_MOESM10_ESM.xlsx", header = 1)

In [3]:
def get_interactions(df):
    interactions = dict()
    for i, row in df.iterrows():
        a = str(row["Protein1_UniProt_Accession"]).strip()
        b = str(row["Protein2_UniProt_Accession"]).strip()
        if a not in interactions:
            interactions[a] = {b}
        else:
            interactions[a].add(b)
        if b not in interactions:
            interactions[b] = {a}
        else:
            interactions[b].add(a)
    return interactions

allowed_interactions = get_interactions(allowed_interactions_source)

In [4]:
def is_fp(accessions_a, accessions_b, interactions):
    for p1 in accessions_a:
        for p2 in accessions_b:
            if p2 in interactions and p1 in interactions[p2]:
                return False
            if p1 in interactions and p2 in interactions[p1]:
                return False
    return True

In [5]:
def eval_result(result, allowed_interactions, accessions_a = "Accession A", accessions_b = "Accession B"):
    interlinks = 0
    intralinks = 0
    fp = 0
    for i, row in result.iterrows():
        a = {x.strip() for x in str(row[accessions_a]).split(";")}
        b = {x.strip() for x in str(row[accessions_b]).split(";")}
        if len(a.intersection(b)) > 0:
            intralinks += 1
        else:
            if is_fp(a, b, allowed_interactions):
                fp += 1
            interlinks += 1

    print(f"Crosslinks: {result.shape[0]}")
    print(f"Intra + Inter: {intralinks + interlinks}")
    print(f"Intralinks: {intralinks}")
    print(f"Interlinks: {interlinks}")
    print(f"Interlinks (true): {interlinks - fp}")
    print(f"Interlinks (false): {fp}")
    print(f"FDR: {fp / (result.shape[0] - fp)}")
    print(f"FDR (inter): {fp / (interlinks - fp)}")

In [6]:
annika_xl = pd.read_excel("MS_Annika/Results/top2_with_mono_Crosslinks_1%_nodecoy.xlsx")
eval_result(annika_xl, allowed_interactions)

Crosslinks: 5134
Intra + Inter: 5134
Intralinks: 3903
Interlinks: 1231
Interlinks (true): 1194
Interlinks (false): 37
FDR: 0.007259172061997253
FDR (inter): 0.03098827470686767


In [7]:
from xiFDRToAnnikaResultConverter import create_annika_result

In [8]:
annika_xifdr_xl1 = create_annika_result("MS_Annika_+_xiFDR/Results/1%_xl_boost_all/boost_all_Links_xiFDR2.2.1.csv")
annika_xifdr_xl2 = create_annika_result("MS_Annika_+_xiFDR/Results/1%_xl_boost_between/boost_between_Links_xiFDR2.2.1.csv")
annika_xifdr_xl3 = create_annika_result("MS_Annika_+_xiFDR/Results/1%_xl_boost_none/boost_none_Links_xiFDR2.2.1.csv")

read 6594 crosslinks from file.
read 6519 crosslinks from file.
read 6375 crosslinks from file.


In [9]:
eval_result(annika_xifdr_xl1, allowed_interactions)

Crosslinks: 6594
Intra + Inter: 6594
Intralinks: 5878
Interlinks: 716
Interlinks (true): 713
Interlinks (false): 3
FDR: 0.0004551661356395084
FDR (inter): 0.004207573632538569


In [10]:
eval_result(annika_xifdr_xl2, allowed_interactions)

Crosslinks: 6519
Intra + Inter: 6519
Intralinks: 5749
Interlinks: 770
Interlinks (true): 767
Interlinks (false): 3
FDR: 0.00046040515653775324
FDR (inter): 0.003911342894393742


In [11]:
eval_result(annika_xifdr_xl3, allowed_interactions)

Crosslinks: 6375
Intra + Inter: 6375
Intralinks: 5878
Interlinks: 497
Interlinks (true): 494
Interlinks (false): 3
FDR: 0.00047080979284369113
FDR (inter): 0.006072874493927126


In [12]:
xifdr_xl1 = create_annika_result("xiSearch_+_xiFDR/Results/1%_xl_boost_all/boost_all_Links_xiFDR2.2.1.csv")
xifdr_xl2 = create_annika_result("xiSearch_+_xiFDR/Results/1%_xl_boost_between/boost_between_Links_xiFDR2.2.1.csv")
xifdr_xl3 = create_annika_result("xiSearch_+_xiFDR/Results/1%_xl_boost_none/boost_none_Links_xiFDR2.2.1.csv")

read 7697 crosslinks from file.
read 6904 crosslinks from file.
read 7589 crosslinks from file.


In [13]:
eval_result(xifdr_xl1, allowed_interactions)

Crosslinks: 7697
Intra + Inter: 7697
Intralinks: 7663
Interlinks: 34
Interlinks (true): 34
Interlinks (false): 0
FDR: 0.0
FDR (inter): 0.0


In [14]:
eval_result(xifdr_xl2, allowed_interactions)

Crosslinks: 6904
Intra + Inter: 6904
Intralinks: 5082
Interlinks: 1822
Interlinks (true): 1795
Interlinks (false): 27
FDR: 0.003926130580194852
FDR (inter): 0.015041782729805013


In [15]:
eval_result(xifdr_xl3, allowed_interactions)

Crosslinks: 7589
Intra + Inter: 7589
Intralinks: 7555
Interlinks: 34
Interlinks (true): 34
Interlinks (false): 0
FDR: 0.0
FDR (inter): 0.0
